In [ ]:
# TFRS와 호환되는 이전 version 설치
# 꼭 다시 시작한 직후 실행하세요!
!pip install --upgrade --force-reinstall -q \
  tensorflow==2.15.0 \
  tensorflow-recommenders==0.7.3 \
  tensorflow-datasets==4.9.4 \
  protobuf==3.20.* \
  ml-dtypes==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━

# 영화 추천 - 검색 모델


실제 추천 시스템은 대개 두 단계로 구성됩니다:

1. **검색 단계(Retrieval stage)**: 모든 가능한 후보군에서 초기 후보 수백 개 선택하는 역할을 합니다. 주요 목표는 사용자가 관심 없어 할 후보들을 효율적으로 걸러내는 것입니다. 검색 모델은 수백만 개의 후보와 다룰 수 있어야 하므로, 계산적으로 효율적이어야 합니다.

2. **순위 단계(Ranking stage)**: 검색 모델의 출력 결과를 세밀하게 조정하여 가능한 최선의 추천 몇 가지를 선택합니다. 이 작업의 목표는 사용자가 관심 있을 법한 아이템 세트를 좁혀서 유력한 후보 목록을 작성하는 것입니다.

이 노트북에서는 첫 번째 단계인 검색에 모델을 구현합니다.

검색 모델은 두 개의 하위 모델로 구성됩니다:

1. **쿼리 모델(Query model)**: 쿼리 feature를 사용하여 쿼리 표현(일반적으로 고정 차원의 임베딩 벡터)을 계산합니다.
2. **후보 모델(Candidate model)**: 후보 feature를 사용하여 후보 표현(동일한 크기의 벡터)을 계산합니다.

그런 다음 두 모델의 출력을 함께 곱하여 쿼리 후보 선호도 점수를 제공하며 점수가 높을수록 후보와 쿼리 간의 더 나은 일치를 나타냅니다.

이 notebook에서는 Movielens 데이터 세트를 사용하여 이러한 two-tower  모델을 구축하고 훈련합니다.

다음을 수행합니다:

1. 데이터를 가져와서 훈련 세트와 테스트 세트로 나눕니다.
2. 검색 모델을 구현합니다.
3. 모델을 피팅하고 평가합니다.
4. 효율적인 서빙을 위해 근사 최근접 이웃(ANN) 인덱스를 구축하여 모델을 내보냅니다.

이러한 과정을 통해 사용자가 관심 있을 가능성이 높은 아이템을 효율적으로 검색하는 모델을 구축하고 평가하는 방법을 배울 수 있습니다.

In [ ]:
import os
# 데이터를 보기 좋게 출력하기 위한 라이브러리
import pprint
# 임시 파일과 디렉토리를 생성하고 관리하기 위한 라이브러리
import tempfile
# 타입 힌트를 위한 라이브러리
from typing import Dict, Text

import numpy as np
import tensorflow as tf
# 다양한 데이터셋을 쉽게 로드할 수 있는 라이브러리
import tensorflow_datasets as tfds
# 추천 시스템을 구축하기 위한 TensorFlow 라이브러리
import tensorflow_recommenders as tfrs

# Version 확인
print("TF:", tf.__version__)
print("TFRS:", tfrs.__version__)
print("TFDS:", tfds.__version__)

TF: 2.15.0
TFRS: v0.7.3
TFDS: 4.9.4


## dataset 준비

-  [Tensorflow Datasets](https://www.tensorflow.org/datasets)의 MovieLens 데이터셋을 사용합니다.  

- movielens/100k_ratings를 로드하면 등급 데이터가 포함된 tf.data.Dataset 객체가 생성되고 movielens/100k_movies를 로드하면 영화 데이터만 포함하는 tf.data.Dataset 객체가 생성됩니다.

- MovieLens 데이터 세트에는 미리 정의된 분할이 없기 때문에 모든 데이터는  `train` split 에 있습니다.

In [ ]:
# 평점 데이터 로드
# MovieLens 100k 데이터셋의 평점 데이터를 "train" 분할로 로드합니다.
# "train" 분할은 모델이 충분히 학습할 수 있을 만큼의 데이터를 포함하고 있습니다.
ratings = tfds.load("movielens/100k-ratings", split="train")

# 사용 가능한 모든 영화의 특성 로드
# MovieLens 100k 데이터셋의 영화 특성 데이터를 "train" 분할로 로드합니다.
movies = tfds.load("movielens/100k-movies", split="train")

# 평점 데이터와 영화 데이터의 길이를 출력합니다.
len(ratings), len(movies)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteHPXPZ8/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


(100000, 1682)

In [ ]:
# ratings 데이터셋에서 1개의 샘플을 가져옵니다.
for x in ratings.take(1).as_numpy_iterator():
    # 가져온 샘플을 넘파이 형식으로 변환한 후 보기 좋게 출력합니다.
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [ ]:
# movies 데이터셋에서 1개의 샘플을 가져옵니다.
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


ratings 데이터세트에서 'user_id' 및 'movie_title' 필드만 사용합니다.  
movies 데이터세트에서는 'movie_title' 필드만 사용합니다.

In [ ]:
# ratings 데이터셋에서 "movie_title"과 "user_id" 열만 선택하여 새로운 데이터셋을 만듭니다.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})

# movies 데이터셋에서 "movie_title" 열만 추출하여 새로운 데이터셋을 만듭니다.
movies = movies.map(lambda x: x["movie_title"])

## Train / Test split  

그러나 여기서는 random 분할을 사용하여 ratings의 80%를 train 세트에 넣고 20%를 테스트 세트에 넣습니다.

In [ ]:
# TensorFlow의 랜덤 시드를 설정하여 재현 가능한 결과를 만듭니다.
tf.random.set_seed(42)

# ratings 데이터셋을 섞습니다.
# 총 100,000개의 데이터를 섞으며, 시드를 42로 설정하여 재현 가능한 결과를 보장합니다.
# reshuffle_each_iteration=False로 설정하여 각 반복마다 데이터를 다시 섞지 않습니다.
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# 섞인 데이터셋에서 처음 80,000개의 데이터를 훈련 데이터셋으로 사용합니다.
train = shuffled.take(80_000)

# 섞인 데이터셋에서 처음 80,000개의 데이터를 건너뛰고, 다음 20,000개의 데이터를 테스트 데이터셋으로 사용합니다.
test = shuffled.skip(80_000).take(20_000)

# 훈련 데이터셋과 테스트 데이터셋의 크기를 출력합니다.
len(train), len(test)

(80000, 20000)

또한 데이터에 존재하는 고유한 사용자 ID와 영화 제목을 알아봅시다.

이는 범주형 특성의 원시 값을 모델의 임베딩 벡터에 매핑할 수 있어야 하기 때문에 중요합니다. 이를 위해서는 원시 특성 값을 연속 범위의 정수로 매핑하는 어휘가 필요합니다. 이를 통해 임베딩 테이블에서 해당 임베딩을 조회할 수 있습니다.

In [ ]:
# 영화 제목 데이터 전체를 배치 처리합니다.
movie_titles = movies.batch(10_000)
# 사용자 ID 데이터 전체를 배치 처리하고, 각 배치에서 "user_id" 키에 해당하는 값을 매핑합니다.
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [ ]:
# 중복을 제거한 영화 제목 목록을 생성합니다.
unique_movie_titles = np.unique(np.array(list(movie_titles)))
# 중복을 제거한 사용자 ID 목록을 생성합니다.
unique_user_ids = np.unique(np.array(list(user_ids)))

# 중복 제거 영화 제목 목록에서 처음 10개를 출력합니다.
unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## 모델 구현

two-tower(Query model + Candidate model) 검색 모델을 구축합니다. 각 타워를 개별적으로 구축한 다음 최종 모델에서 결합합니다.

- 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보 item을 효율적으로 제거하는 것입니다.   
- 가능한 모든 후보 중 수백개의 초기 후보 집합을 선택하여 다음 단계의 순위 네트워크로 넘깁니다.  
- 검색 모델은 수백만개의 후보 아이템을 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*9BVJU6t9XL_GLQU8D_4wGg.png" width=400 />

### query tower

쿼리 및 후보 표현(candidate representation)의 차원을 결정합니다. 더 큰 값을 주면 모델이 더 정확할 수 있지만 학습이 느리고 과적합되는 경향이 더 큽니다.

In [ ]:
# 사용자와 영화의 ID를 임베딩 벡터로 변환할 때 사용할 벡터의 차원을 32로 설정합니다.
embedding_dimension = 32

쿼리 모델 자체를 정의 합니다.   Keras 전처리 레이어를 사용하여 먼저 사용자 ID를 정수로 변환한 다음`Embedding` 레이어를 통해 사용자 임베딩으로 변환합니다. 이전에 구한 사용자 ID 목록을 vocabulary로 사용합니다.  

- StringLookup : string feature를 integer index로 mapping 하는 전처리 레이어

In [ ]:
# StringLookup 레이어를 사용하여 텍스트 데이터를 정수 인덱스로 변환하고,
# 이어서 Embedding 레이어를 사용하여 각 정수 인덱스를 밀집 벡터로 매핑
user_model = tf.keras.Sequential([
  # 고유한 사용자 ID 목록에서 문자열을 조회
  tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
  # 정수 인덱스를 밀집 벡터로 매핑 (unkown token을 위한 추가 임베딩을 포함)
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

이와 같은 간단한 모델은 고전적인 행렬 분해 접근 방식과 정확히 일치합니다.  마지막에 embedding_dimension 전체 출력을 반환하는 한 표준 Keras 구성 요소를 사용하여 임의로 복잡한 모델로 쉽게 확장할 수 있습니다.

### candidate tower

동일한 작업을 candidate tower 에 수행합니다.

In [ ]:
# 영화 제목을 임베딩 벡터로 변환하는 모델
movie_model = tf.keras.Sequential([
  # 영화 제목을 고유한 정수 인덱스로 변환하는 문자열 조회 레이어
  tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token=None),
  # 정수 인덱스를 고차원 임베딩 벡터로 변환하는 임베딩 레이어
  # unknown token을 감안해 영화 제목의 수 + 1 만큼의 크기로 임베딩 레이어를 생성
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics

훈련 데이터에는  positive (user, movie) 쌍이 있습니다. 우리 모델이 얼마나 좋은지 평가하려면, 모델이 계산하는 이 쌍의 선호도 점수를 다른 모든 가능한 후보들의 점수와 비교해야 합니다. 만약 positive 쌍에 대한 점수가 다른 후보들보다 높다면, 우리 모델은 매우 정확하다고 볼 수 있습니다.

이를 위해 `tfrs.metrics.FactorizedTopK` 메트릭을 사용할 수 있습니다. 이 메트릭에는 필수 인자가 하나 있는데, 바로 평가를 위한 암시적 부정 후보들로 사용되는 데이터셋입니다.

우리의 경우, 이것은 movies 데이터셋이며, 우리의 영화 모델을 통해 임베딩으로 변환됩니다.

In [ ]:
# 'movies' 데이터셋의 각 영화를 128개씩 묶어서 배치를 만들고, 'movie_model'을 이용해 영화의 임베딩을 계산합니다.
# 계산된 임베딩들은 'FactorizedTopK' 메트릭의 후보로 사용되어 모델의 성능을 평가하는 데 사용됩니다.
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

### Loss

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 task가 있습니다.

이 경우 **손실 함수와 메트릭 계산**을 함께 묶는 편리한 래퍼인 'Retrieval' 작업 개체를 사용합니다.


In [ ]:
task = tfrs.tasks.Retrieval(metrics=metrics)

task 자체는 쿼리 및 후보 임베딩을 인수로 사용하고 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 모델 구축을 간소화하는 기본 모델 클래스(`tfrs.models.Model`)를 노출합니다. 우리가 해야 할 일은 `__init__` 메소드에서 구성 요소를 설정하고 raw feature를 입력으로 받아 손실 값을 반환하는 `compute_loss` 메소드를 구현하는 것입니다.

그러면 기본 모델이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

In [ ]:
# Movielens 데이터셋을 이용하여 사용자와 영화의 임베딩을 학습하는 추천 시스템 모델 정의
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    # movie_model과 user_model을 각각 Keras 모델로 초기화
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    # 추천 시스템의 학습과 평가를 위한 task를 초기화
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 사용자 특성을 선택하고 user_model에 전달하여 임베딩을 계산
    user_embeddings = self.user_model(features["user_id"])
    # 영화 특성을 선택하고 movie_model에 전달하여 임베딩을 계산
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # task는 손실(loss)과 메트릭(metrics)을 계산
    return self.task(user_embeddings, positive_movie_embeddings)

## Fitting and evaluating

모델을 정의한 후 표준 Keras 피팅 및 평가 루틴을 사용하여 모델을 피팅하고 평가할 수 있습니다.

먼저 모델을 인스턴스화하겠습니다.

In [ ]:
# retrieval model 생성
model = MovielensModel(user_model, movie_model)
# 모델 컴파일 (Adagrad 옵티마이저를 사용하여 학습률을 0.1로 설정)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

그런 다음 훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

In [ ]:
# 훈련 데이터셋을 섞고, 배치 크기를 8192로 설정한 후, 캐시 처리
cached_train = train.shuffle(100_000).batch(8192).cache()

# 테스트 데이터셋의 배치 크기를 4096으로 설정한 후, 캐시 처리
cached_test = test.batch(4096).cache()

모델을 훈련합니다.

In [ ]:
history = model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 28s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0072 - factorized_top_k/top_10_categorical_accuracy: 0.0163 - factorized_top_k/top_50_categorical_accuracy: 0.0906 - factorized_top_k/top_100_categorical_accuracy: 0.1688 - loss: 69838.9084 - regularization_loss: 0.0000e+00 - total_loss: 69838.9084
Epoch 2/3
10/10 [==============================] - 23s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0030 - factorized_top_k/top_5_categorical_accuracy: 0.0203 - factorized_top_k/top_10_categorical_accuracy: 0.0406 - factorized_top_k/top_50_categorical_accuracy: 0.1711 - factorized_top_k/top_100_categorical_accuracy: 0.2953 - loss: 67451.5746 - regularization_loss: 0.0000e+00 - total_loss: 67451.5746
Epoch 3/3
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0237

모델이 학습됨에 따라 손실이 감소하고 top-k 검색 메트릭 세트가 업데이트됩니다. 이는 전체 후보 집합에서 검색된 상위 k 항목에 true positive가 있는지 여부를 알려줍니다. 예를 들어 top-5 categorical accuracy정확도 메트릭이 0.2이면 평균적으로  true positive가 검색된 상위 5개 항목의 20%에 해당한다는 것을 알 수 있습니다.

이 예에서는 평가뿐만 아니라 훈련 중에 메트릭을 평가합니다. 대규모 후보 집합에서는 이 작업이 매우 느릴 수 있으므로 훈련에서는 메트릭 계산을 끄고 평가에서만 실행하는 것이 현명할 수 있습니다.

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 8s 829ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0090 - factorized_top_k/top_10_categorical_accuracy: 0.0198 - factorized_top_k/top_50_categorical_accuracy: 0.1225 - factorized_top_k/top_100_categorical_accuracy: 0.2349 - loss: 31077.8447 - regularization_loss: 0.0000e+00 - total_loss: 31077.8447


{'factorized_top_k/top_1_categorical_accuracy': 0.0010499999625608325,
 'factorized_top_k/top_5_categorical_accuracy': 0.008999999612569809,
 'factorized_top_k/top_10_categorical_accuracy': 0.01979999989271164,
 'factorized_top_k/top_50_categorical_accuracy': 0.12245000153779984,
 'factorized_top_k/top_100_categorical_accuracy': 0.23485000431537628,
 'loss': 28257.314453125,
 'regularization_loss': 0,
 'total_loss': 28257.314453125}

테스트 세트의 성능이 훈련 세트의 성능보다 훨씬 나쁜 이유는 주로 두 가지 요인 때문입니다:

1. **과적합(Overfitting)**: 모델이 훈련 데이터를 외우는 경향이 있기 때문에, 본 데이터에 대해 성능이 더 좋을 가능성이 높습니다. 이러한 과적합 현상은 모델이 많은 파라미터를 가질 때 특히 강하게 나타납니다. 모델 정규화 및 사용자 및 영화 특성의 사용을 통해 모델이 본 적 없는 데이터에 대해 더 잘 일반화할 수 있도록 하여 이를 완화할 수 있습니다.

2. **이미 시청한 영화의 재추천**: 모델이 사용자가 이미 시청한 영화를 다시 추천하는 경우, 이러한 알려진 긍정적인 시청이 테스트 영화를 상위 K개 추천에서 밀어낼 수 있습니다. 이 두 번째 현상은 테스트 추천에서 이전에 본 영화를 제외함으로써 해결할 수 있습니다. 이 접근법은 추천 시스템 문헌에서 비교적 일반적이지만, 이 노트북에서는 고려하지 않습니다. 과거의 시청 이력과 맥락 정보로부터 모델이 자동으로 이러한 행동을 배울 것으로 예상하므로, 과거에 시청한 영화를 추천하지 않는 것이 중요하다면, 적절하게 지정된 모델이 이를 자동으로 학습할 것으로 기대해야 합니다. 추가적으로, 동일한 항목을 여러 번 추천하는 것(예: 언제나 인기 있는 TV 시리즈나 정기적으로 구매하는 상품)이 종종 적절할 수도 있습니다.

요약하자면, 모델이 훈련 데이터에 과적합되어 테스트 데이터에 대한 성능이 떨어지는 현상과, 이미 시청한 영화를 다시 추천하는 것을 피하기 위한 전략에 대해 설명하고 있습니다. 모델이 과거 사용자의 행동과 맥락 정보를 바탕으로 스스로 학습하여 이러한 문제를 해결할 수 있도록 하는 것이 중요하며, 때로는 동일한 아이템을 여러 번 추천하는 것이 사용자에게 유용할 수 있음을 강조하고 있습니다.

## 예측하기
이제 모델이 있으므로 예측을 할 수 있습니다. 이를 위해 tfrs.layers.factorized_top_k.BruteForce 레이어를 사용할 수 있습니다.

index.index_from_dataset 함수는 전체 영화 데이터 세트를 통해 영화 임베딩을 생성하고 이를 인덱스에 추가하여, 사용자 쿼리에 대한 영화 추천을 가능하게 하는 역할을 합니다. 이 함수는 주로 두 개의 파라미터를 받습니다.  하나는 후보 영화의 배치(batch), 다른 하나는 해당 영화를 임베딩으로 변환하는 모델입니다, 이 과정을 통해, 모델이 사용자 모델을 통해 생성된 쿼리 임베딩과 영화 모델을 통해 생성된 영화 임베딩 사이의 관계를 학습하고, 사용자에게 적합한 영화를 빠르게 검색하여 추천할 수 있습니다​​.

In [ ]:
# 원시 쿼리 feature를 사용하고 전체 영화 데이터 세트에서 영화를 추천하는 모델을 생성
brute_index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# 영화 데이터셋을 묶어서 인덱스를 생성합니다. 여기서는 각 영화를 100개씩 배치로 처리하고,
# 영화 모델을 매핑하여 영화 임베딩을 얻습니다.
brute_index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# 사용자 ID "42"에 대한 영화 추천을 실행합니다.
_, titles = brute_index(tf.constant(["42"]))
# 사용자 42에 대한 상위 3개 영화 추천을 출력합니다.
print(f"사용자 42에 대한 추천: {titles[0, :3]}")

사용자 42에 대한 추천: [b"Kid in King Arthur's Court, A (1995)" b'Rent-a-Kid (1995)'
 b'Homeward Bound: The Incredible Journey (1993)']
